In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="=https://dagshub.com/HazSyl1/Cancer_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="=HazSyl1"
os.environ["MLFLOW_TRACKING_PASSWORD"]="=a7a47b400fa96a13114cdb46a08acb04513b3b29"


In [18]:
pip show mlflow

Name: mlflow
Version: 2.2.2
Summary: MLflow: A Platform for ML Development and Productionization
Home-page: https://mlflow.org/
Author: Databricks
Author-email: 
License: Apache License 2.0
Location: c:\users\hazsy\anaconda3\envs\cancer\lib\site-packages
Requires: alembic, click, cloudpickle, databricks-cli, docker, entrypoints, Flask, gitpython, importlib-metadata, Jinja2, markdown, matplotlib, numpy, packaging, pandas, protobuf, pyarrow, pytz, pyyaml, querystring-parser, requests, scikit-learn, scipy, shap, sqlalchemy, sqlparse, waitress
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf


In [7]:
model =tf.keras.models.load_model("artifacts/training/model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml , create_directories , save_json

In [10]:
class ConfigurationManager():
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
                 self.config =read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data/",
            mlflow_uri="https://dagshub.com/HazSyl1/Cancer_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
        
        
                
        

In [11]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'

In [12]:
#pip install mlflow==2.2.2

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras 
from urllib.parse import urlparse

In [24]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config =config
        
    def _valid_generator(self):
        
        datagenerator_kwargs=dict(
            rescale = 1./255,
            validation_split=0.30
        )
        
        dataflow_kwargs=dict(
            target_size =self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear")
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path:Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores={"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme 

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )     
            if tracking_url_type_store!= "file":
                mlflow.keras.log_model(self.model , "model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model , "model")
    
        
        
    
    
    

In [23]:
try:
    config=ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e 

[2024-01-05 00:08:14,833]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-05 00:08:14,840]:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-05 00:08:14,845]:INFO:common:created directory at: artifacts]


Found 102 images belonging to 2 classes.
7/7 [==============================] - 30s 4s/step - loss: 26.8013 - accuracy: 0.4314
[2024-01-05 00:08:46,360]:INFO:common:json file saved at: scores.json]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'S:\\'

In [19]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'